This is a data exploration and analysis of UCI Worldcup Downhill racing stats. With most of the data comming from pro
mountian bike rider Elliot Jacksons's site http://worldcup.eliotjackson.com/

The intial goal of my data exploration and analysis of these races is to try and determine what factors lead to someone
winning a race and in the end I hope to build a prediction model for who will win race results based off of my findings
To do all this work I will use all the tools I have build through-out CS251

In [1]:
# importing all needed librarys and setting intial parameters afor the notebook
import os
import random
import numpy as np
import cupy as cp
import matplotlib.pyplot as plt
import pandas as pd

#imports for the tools I created
import rbf_net
import knn_P7 as knn
import kmeansGPU as kmeans
import linear_regression_gpu as lineaer_regeression
import naive_bayes_multinomial_P7 as naive_bayes

# importing libraries for scrapping webdata
import requests
from bs4 import BeautifulSoup , NavigableString
import json
from requests_html import HTMLSession,AsyncHTMLSession


plt.style.use(['seaborn-colorblind', 'seaborn-darkgrid'])
plt.rcParams.update({'font.size': 20})

np.set_printoptions(suppress=True, precision=5)

# Automatically reload external modules
%load_ext autoreload
%autoreload 2


To start I need to gather the data and clean it before I can do the data analysis.
 (From: http://worldcup.eliotjackson.com)

I am going to collect all the data from the site. This turned out to be quite a process so I will run though
how I gathered the data and structured it below

To start I use the chrome developer window and help from friends to learn that I needed to eb searhing for the jsons in
the api calls for the site (using the requests package to make calls to the different urls and read in the data from them as a json file)
Also when the url call returned only a single json file (and not nested json Files) I was able to use pandas.read_json()

Creating a pandas database of all the riders


In [2]:
riders_df = pd.read_json('http://worldcup.eliotjackson.com/api/riders')
#TODO make ride names the index of the data frame



Collecting all the data from the races category

In [3]:
venues_df = pd.read_json('http://worldcup.eliotjackson.com/api/races/venues')
venues_df

,date,qualy_date,practice_date,year,event,link,uci_link,location,country_code,round_num,...,juniorwomen_qualy_racetime,elitemen_final_racetime,elitewomen_final_racetime,juniormen_final_racetime,juniorwomen_final_racetime,utc_practice_date,utc_final_date,redbull_link,rounds,current_round
0,2020-09-07,2020/09/06,2019/09/05,2020,World Championship,https://www.rootsandrain.com/race6987/2019-sep...,None,Austria,AUT,World Championship,...,2020-09-06 10:15~2020-09-06 10:30,2020-09-07 11:30~2020-09-07 15:00,2020-09-07 10:30~2020-09-07 11:30,2020-09-07 08:15~2020-09-07 09:00,2020-09-07 07:45~2020-09-07 08:15,2019/09/05,2020/09/07,None,2.0,1.0
1,2020-10-16,2020/10/15,None,2020,World Cup,https://www.rootsandrain.com/race6987/2019-sep...,https://prod.chronorace.be/api/results/uci/dh/...,Slovenia,SLO,1,...,2020-10-15 10:15~2020-10-15 10:30,2020-10-16 11:30~2020-10-16 15:00,2020-10-16 10:30~2020-10-16 11:30,2020-10-16 08:15~2020-10-16 09:00,2020-10-16 07:45~2020-10-16 08:15,2019/09/05,2020/10/16,None,2.0,1.0
2,2020-10-18,2020/10/17,None,2020,World Cup,https://www.rootsandrain.com/race6987/2019-sep...,https://prod.chronorace.be/api/results/uci/dh/...,Slovenia,SLO,2,...,2020-10-17 10:15~2020-10-17 10:30,2020-10-18 11:30~2020-10-18 15:00,2020-10-18 10:30~2020-10-18 11:30,2020-10-18 08:15~2020-10-18 09:00,2020-10-18 07:45~2020-10-18 08:15,2019/09/05,2020/10/18,None,2.0,1.0
3,2019-08-31,2019/08/30,None,2019,World Championship,https://www.rootsandrain.com/race6988/2019-sep...,http://prod.chronorace.be/api/results/uci/dh/r...,Canada,CAN,World Championship,...,2019-08-30 14:45~2019-08-30 15:00,2019-08-31 16:00~2019-08-31 19:30,2019-08-31 15:00~2019-08-31 16:00,2019-08-31 12:45~2019-08-31 13:30,2019-08-31 12:15~2019-08-31 12:45,None,2019/08/31,None,8.0,8.0
4,2019-04-28,2019/04/27,2019/04/26,2019,World Cup,https://www.rootsandrain.com/race6980/2019-apr...,http://prod.chronorace.be/api/results/uci/dh/r...,Slovenia,SLO,1,...,2019-04-27 10:15~2019-04-27 10:30,2019-04-28 11:30~2019-04-28 15:00,2019-04-28 10:30~2019-04-28 11:30,2019-04-28 08:15~2019-04-28 09:00,2019-04-28 07:45~2019-04-28 08:15,2019/04/26,2019/04/28,None,8.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,1995-09-23,1995/09/22,None,1995,World Championship,https://www.rootsandrain.com/race1459/1995-sep...,None,Germany,GER,World Championship,...,None,None,None,None,None,None,None,None,NaN,NaN
188,1994-09-18,1994/09/17,None,1994,World Championship,https://www.rootsandrain.com/race2263/1994-sep...,None,United States,USA,World Championship,...,None,None,None,None,None,None,None,None,NaN,NaN
189,1993-09-19,1993/09/18,None,1993,World Championship,https://www.rootsandrain.com/race2264/1993-sep...,None,France,FRA,World Championship,...,None,None,None,None,None,None,None,None,NaN,NaN
190,1992-09-20,1992/09/19,None,1992,World Championship,https://www.rootsandrain.com/race2265/1992-sep...,None,Canada,CAN,World Championship,...,None,None,None,None,None,None,None,None,NaN,NaN


From here I am going to run this script below to loop collect every json file holding the results of each race

In [4]:
#TODO make a check to see if files have already been grabbed
# for venue , year in zip(venues_df.venue,venues_df.year):
#     with open(f'data/results_jsons/{year}_{venue}_results.json','w') as f:
#         result_request = requests.get(url=f'http://mworldcup.eliotjackson.com/api/races/results?year={year}&venue={venue}')
#         result_json = result_request.json()
#         json.dump(result_json,f)
#         print(f'Finished getting: {year}_{venue}_results')

In [5]:
# Now I am going to save the riders json file in data/riders
# wc_api_riders_request = requests.get('http://mworldcup.eliotjackson.com/api/riders')
# wc_api_riders_json = wc_api_riders_request.json()
# with open('data/riders/wc_riders.json','w') as f:
#     json.dump(wc_api_riders_json, f)




Now I am going to look at the results from the elite mens category during the 2019 Snowshoe WC



In [10]:
#Helper function for load_race_result_json_to_df to get each categories data
def get_race_cat_results_df(race_cat,race_results_json):
    #initial conditions
    race_days = ['timed','qualy','final']

    race_cat_timed_df = pd.json_normalize(race_results_json[race_cat]['timed'])
    race_cat_qualy_df = pd.json_normalize(race_results_json[race_cat]['qualy'])
    race_cat_final_df = pd.json_normalize(race_results_json[race_cat]['final'])

    # #put all race cat dfs in a list
    # race_cats_df_list = [race_cat_timed_df,race_cat_qualy_df,race_cat_final_df]
    # race_cats_df_list = list(map(lambda df : df['uci_id'].astype('int'),iter(race_cats_df_list)))
    # race_cats_df_list = list(map(lambda df : df.set_index(['uci_id'],inplace=True),iter(race_cats_df_list)))
    # variables to drop for all data sets

    #set all dataframes idexs to races unique uci_id number and make uci_id an int
    race_cat_timed_df['uci_id'].astype('int')
    race_cat_qualy_df['uci_id'].astype('int')
    race_cat_final_df['uci_id'].astype('int')

    race_cat_timed_df.set_index(['uci_id'],inplace=True)
    race_cat_qualy_df.set_index(['uci_id'],inplace=True)
    race_cat_final_df.set_index(['uci_id'],inplace=True)

    #hold variables that are for the whole race event in a dict
    race_event_dict = {}

    #get the date of each day
    timed_date = race_cat_timed_df.pop('date').unique()
    qualy_date = race_cat_qualy_df.pop('date').unique()
    final_date = race_cat_final_df.pop('date').unique()

    timed_date = timed_date[0]
    qualy_date = qualy_date[0]
    final_date = final_date[0]

    race_event_dict['race_dates'] = [timed_date,qualy_date,final_date]

    #get round number and venue
    race_venue = race_cat_final_df.pop('venue')
    race_year = race_cat_final_df.pop('year') # maybe just drop not store since have race dates
    race_round_num = race_cat_final_df.pop('round_num')

    race_event_dict['race_venue'] = race_venue
    race_event_dict['race_year'] = race_year
    race_event_dict['race_round_num'] = race_round_num

    #drop variables that repeat and are same in all DFs for timed and qualy
    race_cat_timed_df.drop(columns = ['round_num','venue','year','uci_code'],inplace=True)
    race_cat_qualy_df.drop(columns = ['round_num','venue','year','uci_code'],inplace=True)
    race_cat_final_df.drop(columns = 'uci_code',inplace=True)

    #rearange the columns in the dataframes
    #t_l is the labels for timed df
    t_l = race_cat_timed_df.columns.values.tolist()
    #changing timed df cols to name, yob,country,junior,bike,team,event,protect,overal,pos,points
    timed_cols_final = [t_l[4]]+[t_l[9]]+[t_l[7]]+[t_l[0]]+[t_l[6]]+[t_l[5]]+[t_l[11]] +t_l[2:4]+[t_l[1]]+[t_l[8]]+[t_l[10]]+t_l[12:]
    race_cat_timed_df = race_cat_timed_df.reindex(columns = timed_cols_final)

    #f_l is final labels and q_l is qualy labels for cols
    q_l = race_cat_qualy_df.columns.values.tolist()
    f_l = race_cat_final_df.columns.values.tolist()

    qualy_cols_final = [q_l[5]]+[q_l[4]]+[q_l[8]]+[q_l[0]]+[q_l[7]]+[q_l[6]]+[q_l[13]]+[q_l[2]]+[q_l[3]]+q_l[9:13]+q_l[14:]
    final_cols_final = [f_l[5]]+[f_l[4]]+[f_l[8]]+[f_l[0]]+[f_l[7]]+[f_l[6]]+[f_l[13]]+[f_l[2]]+[f_l[3]]+f_l[9:13]+f_l[14:]

    race_cat_qualy_df = race_cat_qualy_df.reindex(columns = qualy_cols_final)
    race_cat_final_df = race_cat_final_df.reindex(columns = final_cols_final)

    #list of expected event types
    expected_events_list = ['WC-_timed','WC-_qualy_result','WC-_final_result']

    #pop event series from each dataframe
    events_list = []
    events_list.append(race_cat_timed_df.pop('event').unique().astype('str')[0])
    events_list.append(race_cat_qualy_df.pop('event').unique().astype('str')[0])
    events_list.append(race_cat_final_df.pop('event').unique().astype('str')[0])

    # see if the events from race dfs are in expected df
    for expected_event, event_of_df in zip(expected_events_list,events_list):
        if event_of_df != expected_event:
            print(f'Warning Event {event_of_df} is not expected event of {expected_event}')

    # list to hold timed practice (t)
    # qualifying (q) and finals (f)
    #TODO maybe a cool way to multithred
    event_prefix_list = ['t','q','f'] #for now just setting without loop

    race_cat_timed_df.columns.values[6:].str.a
    print(f'Done Testing Race Cat Function')

# Main function to load in the race results data (employs helper functions above)
def load_race_result_json_to_df(file_path):

    #initial conditions
    race_categories = ['elitemen','elitewomen','juniormen','juniorwomen']

    with open(file_path,'r') as f:
        race_result_json = json.load(f)

        #loop throught each category and get each race category
        #TODO multiprocess/thread later
        for race_cat in race_categories:
            race_cat_df = get_race_cat_results_df(race_cat,race_result_json)


        print(f'Done Testing Main Function')

    return 1

snow_2019_file_path = 'data/results_jsons/2019_Snowshoe_results.json'


snow_2019_df = load_race_result_json_to_df(snow_2019_file_path)

KeyboardInterrupt: 

Now so I can analysise riders that rode in time practice, qualifying, and the finals I will
filter the timed,qualy, and final sub dicts for the elite mens field for riders that rode in all 3


In [106]:
#make elite mens category dfs for timed practice, qualifying, and finals run
snow_2019_elite_men_pract_df = pd.json_normalize(snow_2019_json['elitemen']['timed'])
snow_2019_elite_men_qualy_df = pd.json_normalize(snow_2019_json['elitemen']['qualy'])
snow_2019_elite_men_final_df = pd.json_normalize(snow_2019_json['elitemen']['final'])

Now I am going to look at the info of each dataset one by one

Snowshoe 2019 Elite Men Practice

In [91]:
snow_2019_elite_men_pract_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Columns: 395 entries, date to finish.from_run
dtypes: float64(67), int64(68), object(260)
memory usage: 154.4+ KB


In [92]:
snow_2019_elite_men_pract_df.describe()


,year,round_num,position,overall,points,yob,run1.avg_speed.speed,run1.avg_speed.speed_position,run1.sector1.position,run1.sector1.time_lost_position,...,split3.position,split3.time_behind_position,sector4.position,sector4.time_lost_position,split4.position,split4.time_behind_position,sector5.position,sector5.time_lost_position,finish.position,finish.gap_position
count,50.0,50.0,50.00000,50.00000,50.000000,50.000000,25.000000,25.000000,25.000000,25.000000,...,50.00000,50.00000,50.00000,50.00000,50.00000,50.00000,50.00000,50.00000,50.00000,50.00000
mean,2019.0,8.0,25.50000,29.24000,63.600000,1993.220000,27.276160,13.000000,13.000000,13.000000,...,25.50000,25.50000,25.50000,25.50000,25.50000,25.50000,25.50000,25.50000,25.50000,25.50000
std,0.0,0.0,14.57738,17.55058,33.317275,4.185934,11.210375,7.359801,7.359801,7.359801,...,14.57738,14.57738,14.57738,14.57738,14.57738,14.57738,14.57738,14.57738,14.57738,14.57738
min,2019.0,8.0,1.00000,1.00000,31.000000,1981.000000,4.737000,1.000000,1.000000,1.000000,...,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000
25%,2019.0,8.0,13.25000,14.25000,43.250000,1991.000000,21.176000,7.000000,7.000000,7.000000,...,13.25000,13.25000,13.25000,13.25000,13.25000,13.25000,13.25000,13.25000,13.25000,13.25000
50%,2019.0,8.0,25.50000,28.50000,55.500000,1994.000000,33.857000,13.000000,13.000000,13.000000,...,25.50000,25.50000,25.50000,25.50000,25.50000,25.50000,25.50000,25.50000,25.50000,25.50000
75%,2019.0,8.0,37.75000,42.75000,67.750000,1996.000000,34.892000,19.000000,19.000000,19.000000,...,37.75000,37.75000,37.75000,37.75000,37.75000,37.75000,37.75000,37.75000,37.75000,37.75000
max,2019.0,8.0,50.00000,60.00000,200.000000,2000.000000,36.187000,25.000000,25.000000,25.000000,...,50.00000,50.00000,50.00000,50.00000,50.00000,50.00000,50.00000,50.00000,50.00000,50.00000


In [93]:
snow_2019_elite_men_pract_dtypes = snow_2019_elite_men_pract_df.dtypes
print(f'Prac Dtypes:\n{snow_2019_elite_men_pract_dtypes}')


Prac Dtypes:
date                   object
venue                  object
year                    int64
round_num               int64
junior                 object
                        ...  
finish.time            object
finish.position         int64
finish.gap             object
finish.gap_position     int64
finish.from_run        object
Length: 395, dtype: object


Snowshoe 2019 Elite Men Qualifying:

In [94]:
snow_2019_elite_men_qualy_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89 entries, 0 to 88
Data columns (total 73 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   date                         89 non-null     object 
 1   venue                        89 non-null     object 
 2   year                         89 non-null     int64  
 3   round_num                    89 non-null     int64  
 4   junior                       0 non-null      object 
 5   position                     89 non-null     object 
 6   protected                    19 non-null     object 
 7   overall                      70 non-null     float64
 8   uci_code                     89 non-null     object 
 9   uci_id                       89 non-null     object 
 10  yob                          89 non-null     int64  
 11  name                         89 non-null     object 
 12  team                         66 non-null     object 
 13  bike                  

In [95]:
snow_2019_elite_men_qualy_df.describe()

,year,round_num,overall,yob,points,avg_speed.speed,avg_speed.speed_position,sector1.position,sector1.time_lost_position,split1.position,...,split3.position,split3.time_behind_position,sector4.position,sector4.time_lost_position,split4.position,split4.time_behind_position,sector5.position,sector5.time_lost_position,finish.gap_position,ranking.overall
count,89.0,89.0,70.000000,89.000000,89.0,78.000000,78.000000,78.000000,78.000000,78.000000,...,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000,70.000000
mean,2019.0,8.0,40.842857,1993.539326,0.0,35.350692,39.461538,39.487179,39.487179,39.487179,...,39.500000,39.500000,39.487179,39.487179,39.500000,39.500000,39.448718,39.448718,39.500000,40.842857
std,0.0,0.0,25.526704,4.332898,0.0,2.500670,22.704020,22.650790,22.650790,22.650790,...,22.660538,22.660538,22.665119,22.665119,22.660538,22.660538,22.663345,22.663345,22.660538,25.526704
min,2019.0,8.0,1.000000,1981.000000,0.0,20.551000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,2019.0,8.0,19.500000,1991.000000,0.0,35.261750,20.250000,20.250000,20.250000,20.250000,...,20.250000,20.250000,20.250000,20.250000,20.250000,20.250000,20.250000,20.250000,20.250000,19.500000
50%,2019.0,8.0,37.500000,1994.000000,0.0,35.899000,39.500000,39.500000,39.500000,39.500000,...,39.500000,39.500000,39.500000,39.500000,39.500000,39.500000,39.500000,39.500000,39.500000,37.500000
75%,2019.0,8.0,60.750000,1997.000000,0.0,36.398750,58.750000,58.750000,58.750000,58.750000,...,58.750000,58.750000,58.750000,58.750000,58.750000,58.750000,58.750000,58.750000,58.750000,60.750000
max,2019.0,8.0,97.000000,2000.000000,0.0,37.937000,78.000000,78.000000,78.000000,78.000000,...,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000,97.000000


In [96]:
snow_2019_elite_men_qualy_dtypes = snow_2019_elite_men_qualy_df.dtypes
print(f'Qualy Dtypes:\n{snow_2019_elite_men_qualy_dtypes}')


Qualy Dtypes:
date                    object
venue                   object
year                     int64
round_num                int64
junior                  object
                        ...   
finish.time             object
finish.gap              object
finish.gap_position    float64
ranking.overall        float64
ranking.protected       object
Length: 73, dtype: object


Snowshoe 2019 Elite Men Final:

In [97]:
snow_2019_elite_men_final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61 entries, 0 to 60
Data columns (total 65 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   date                         61 non-null     object 
 1   venue                        61 non-null     object 
 2   year                         61 non-null     int64  
 3   round_num                    61 non-null     int64  
 4   junior                       0 non-null      object 
 5   position                     61 non-null     object 
 6   protected                    18 non-null     object 
 7   overall                      58 non-null     float64
 8   uci_code                     61 non-null     object 
 9   uci_id                       61 non-null     object 
 10  yob                          61 non-null     int64  
 11  name                         61 non-null     object 
 12  team                         54 non-null     object 
 13  bike                  

In [98]:
snow_2019_elite_men_final_df.describe()

,year,round_num,overall,yob,points,avg_speed.speed,avg_speed.speed_position,sector1.position,sector1.time_lost_position,split1.position,...,split3.position,split3.time_behind_position,sector4.position,sector4.time_lost_position,split4.position,split4.time_behind_position,sector5.position,sector5.time_lost_position,finish.gap_position,ranking.overall
count,61.0,61.0,58.000000,61.000000,61.000000,60.000000,60.000000,60.000000,60.000000,60.000000,...,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,58.000000
mean,2019.0,8.0,38.155172,1992.918033,46.606557,37.601983,30.500000,30.500000,30.500000,30.500000,...,30.500000,30.500000,30.500000,30.500000,30.500000,30.500000,30.483333,30.483333,30.500000,38.155172
std,0.0,0.0,24.348206,4.071425,50.396851,0.896575,17.464249,17.464249,17.464249,17.464249,...,17.464249,17.464249,17.464249,17.464249,17.464249,17.464249,17.437046,17.437046,17.464249,24.348206
min,2019.0,8.0,1.000000,1981.000000,0.000000,34.369000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,2019.0,8.0,18.250000,1990.000000,15.000000,37.113500,15.750000,15.750000,15.750000,15.750000,...,15.750000,15.750000,15.750000,15.750000,15.750000,15.750000,15.750000,15.750000,15.750000,18.250000
50%,2019.0,8.0,34.500000,1994.000000,30.000000,37.553000,30.500000,30.500000,30.500000,30.500000,...,30.500000,30.500000,30.500000,30.500000,30.500000,30.500000,30.500000,30.500000,30.500000,34.500000
75%,2019.0,8.0,59.750000,1996.000000,54.000000,38.129750,45.250000,45.250000,45.250000,45.250000,...,45.250000,45.250000,45.250000,45.250000,45.250000,45.250000,45.250000,45.250000,45.250000,59.750000
max,2019.0,8.0,84.000000,1999.000000,250.000000,39.563000,60.000000,60.000000,60.000000,60.000000,...,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,84.000000


In [99]:
snow_2019_elite_men_final_dtypes = snow_2019_elite_men_final_df.dtypes
print(f'Final Dtypes:\n{snow_2019_elite_men_final_dtypes}')

Final Dtypes:
date                    object
venue                   object
year                     int64
round_num                int64
junior                  object
                        ...   
finish.time             object
finish.gap              object
finish.gap_position    float64
ranking.overall        float64
ranking.protected       object
Length: 65, dtype: object


Now I am going to clean up my data starting with the data

In [ ]:
# seting the indexes of all the data frames to to riders individual uci_id

In [107]:
snow_2019_elite_men_qualy_df['uci_id'].isin(snow_2019_elite_men_final_df['uci_id'])

0      True
1      True
2      True
3      True
4      True
      ...  
84    False
85    False
86    False
87    False
88    False
Name: uci_id, Length: 89, dtype: bool

In [100]:
#poping of the event column and appending it to the column name of every other feature in the dataset
# for practice, qualifying and finals.
prac_event = snow_2019_elite_men_pract_df.pop('event')
qualy_event = snow_2019_elite_men_qualy_df.pop('event')
final_event = snow_2019_elite_men_final_df.pop('event')

In [101]:
prac_event = prac_event.astype('str')[0]
qualy_event = qualy_event.astype('str')[0]
final_event = final_event.astype('str')[0]

In [104]:
snow_2019_elite_men_final_df.columns.map(lambda x: f'finals.{x}')

Index(['final.date', 'final.venue', 'final.year', 'final.round_num',
       'final.junior', 'final.position', 'final.protected', 'final.overall',
       'final.uci_code', 'final.uci_id', 'final.yob', 'final.name',
       'final.team', 'final.bike', 'final.country', 'final.dns', 'final.dnf',
       'final.dsq', 'final.points', 'final.speed.speed',
       'final.speed.position', 'final.avg_speed.speed',
       'final.avg_speed.speed_position', 'final.sector1.time',
       'final.sector1.position', 'final.sector1.time_lost',
       'final.sector1.time_lost_position', 'final.split1.time',
       'final.split1.position', 'final.split1.time_behind',
       'final.split1.time_behind_position', 'final.sector2.time',
       'final.sector2.position', 'final.sector2.time_lost',
       'final.sector2.time_lost_position', 'final.split2.time',
       'final.split2.position', 'final.split2.time_behind',
       'final.split2.time_behind_position', 'final.sector3.time',
       'final.sector3.position',

In [ ]:


#dropping the round_num, date
a_final_test = snow_2019_elite_men_final_df.columns.values


#setting event pre fixes for practice, qualy and final

#tests

In [56]:
snow_2019_elite_men_final_df.columns.values

array(['date', 'venue', 'year', 'round_num', 'junior', 'position',
       'protected', 'overall', 'uci_code', 'uci_id', 'yob', 'name',
       'team', 'bike', 'country', 'dns', 'dnf', 'dsq', 'points', 'event',
       'speed.speed', 'speed.position', 'avg_speed.speed',
       'avg_speed.speed_position', 'sector1.time', 'sector1.position',
       'sector1.time_lost', 'sector1.time_lost_position', 'split1.time',
       'split1.position', 'split1.time_behind',
       'split1.time_behind_position', 'sector2.time', 'sector2.position',
       'sector2.time_lost', 'sector2.time_lost_position', 'split2.time',
       'split2.position', 'split2.time_behind',
       'split2.time_behind_position', 'sector3.time', 'sector3.position',
       'sector3.time_lost', 'sector3.time_lost_position', 'split3.time',
       'split3.position', 'split3.time_behind',
       'split3.time_behind_position', 'sector4.time', 'sector4.position',
       'sector4.time_lost', 'sector4.time_lost_position', 'split4.time',
   

In [59]:
event = event.unique()

End of my final finished nopebook code and scratch

In [ ]:
print(f'This is the end of the final code')

In [ ]:
# #set the dataframes indxs to be the uci ids of every rider
# snow_2019_elite_men_pract_df.set_index('uci_id',inplace=True)
# snow_2019_elite_men_qualy_df.set_index('uci_id',inplace=True)
# snow_2019_elite_men_final_df.set_index('uci_id',inplace=True)

In [13]:
final_types = snow_2019_elite_men_final_df.dtypes

In [10]:
a0 = pd.join(snow_2019_elite_men_final_df,snow_2019_elite_men_qualy_df, how = 'inner')
# snow_2019_elite_men_merged = pd.merge(snow_2019_elite_men_final_df,snow_2019_elite_men_qualy_df)

AttributeError: module 'pandas' has no attribute 'join'

In [32]:
# snow_2019_uci_ids_practice = snow_2019_elite_men_practice_df.uci_id.astype('int')
# #getting all the riders that rode in timed practice, qualy, and finals
# snow_2019_uci_ids_qualy = snow_2019_elite_men_qualy_df.uci_id.astype('int')
# snow_2019_uci_ids_final = snow_2019_elite_men_final_df.uci_id.astype('int')

In [33]:
#mergeing all the data frames so that they contain all the riders that rode in timed practice, qualifying, and finals run
snow_2019_elite_men_merged = pd.merge(snow_2019_elite_men_final_df,snow_2019_elite_men_qualy_df)

In [ ]:
# snowshow_2019_json = json.load('data/results_jsons/2019_Snowshoe_results.json')
# snowshow_2019_df = pd.read_json('data/results_jsons/2019_Snowshoe_results.json')

Writing out the psuedo code to pull all info i need


MOST CURRENT WORK TESTIN OUT REQUESTS-HTML



In [21]:
session = HTMLSession()

#Initilizing session object

In [22]:
results_get_request = session.get(url='http://worldcup.eliotjackson.com/results')

In [23]:
#render the page
results_get_request.html.render()

RuntimeError: Cannot use HTMLSession within an existing event loop. Use AsyncHTMLSession instead.

Scratch

In [ ]:
venues_request = requests.get(url='http://worldcup.eliotjackson.com/api/races/venues')
venues_json = venues_request.json()

In [ ]:
maribor2020_results_req = requests.get(url='http://mworldcup.eliotjackson.com/api/races/results?year=2020&venue=Maribor')
maribor2020_results_json = maribor2020_results_req.json()


pandas_maribor2020 = pd.read_json('http://mworldcup.eliotjackson.com/api/races/results?year=2020&venue=Maribor')

In [2]:
race_results2020_request = requests.get(url='http://worldcup.eliotjackson.com/api/results/2020')

In [3]:
results2020_json  = race_results2020_request.json()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [10]:
venues_request = requests.get(url='http://worldcup.eliotjackson.com/api/venues')
venues_html  = venues_request.text

In [9]:
api_request = requests.get(url='http://worldcup.eliotjackson.com/api/riders')
races_request = requests.get(url='http://worldcup.eliotjackson.com/api/races/')

In [12]:
riders_results_request = requests.get(url='http://worldcup.eliotjackson.com/api/riders/results')

In [3]:
maribor2020_results_req = requests.get(url='http://mworldcup.eliotjackson.com/api/races/results?year=2020&venue=Maribor')
maribor2020_results_json = maribor2020_results_req.json()


pandas_maribor2020 = pd.read_json('http://mworldcup.eliotjackson.com/api/races/results?year=2020&venue=Maribor')

In [25]:
wc_stats_api_request = requests.get(url='http://worldcup.eliotjackson.com/api')

In [28]:
wc_api_soup = BeautifulSoup(wc_stats_api_request.text, 'html.parser')
wc_api_html_pretty = wc_api_soup.prettify()

In [30]:
wc_results_main = requests.get(url='http://worldcup.eliotjackson.com/results/')

In [32]:
wc_results_soup = BeautifulSoup(wc_results_main.text,'html.parser')
restults_t = wc_results_soup.prettify()